In [2]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install soundfile
!pip install moviepy


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-wwhppl_y
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-wwhppl_y
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import whisper
import torch
from moviepy.editor import VideoFileClip

def transcribe_audio_files(audio_files, output_txt_path, output_srt_path):
    """
    Verilen ses dosyalarını metin haline dönüştürür ve sonuçları hem .txt hem de .srt dosyalarına kaydeder.

    :param audio_files: Ses dosyalarının yollarını içeren liste
    :param output_txt_path: Metin dosyasının kaydedileceği yol
    :param output_srt_path: Altyazı dosyasının kaydedileceği yol
    :return: None
    """
    # Whisper modelini yükle
    model = whisper.load_model("base")

    # Sonuçları saklamak için boş bir liste
    txt_results = []
    srt_results = []
    srt_counter = 1

    # Her bir ses dosyasını işleyin
    for audio_file in audio_files:
        if audio_file.endswith('.mp4'):
            # MP4 dosyasını işleme
            video = VideoFileClip(audio_file)
            audio = video.audio
            audio_file_path = "temp_audio.wav"
            audio.write_audiofile(audio_file_path)
            result = model.transcribe(audio_file_path)
            txt_results.append(f"MP4 Results:\n{result['text']}\n")
            srt_results.extend(generate_srt(result['segments'], srt_counter))
            srt_counter = len(srt_results) + 1
        elif audio_file.endswith('.mp3'):
            # MP3 dosyasını işleme
            result = model.transcribe(audio_file)
            txt_results.append(f"MP3 Results:\n{result['text']}\n")
            srt_results.extend(generate_srt(result['segments'], srt_counter))
            srt_counter = len(srt_results) + 1
        elif audio_file.endswith('.wav'):
            # WAV dosyasını işleme
            result = model.transcribe(audio_file)
            txt_results.append(f"WAV Results:\n{result['text']}\n")
            srt_results.extend(generate_srt(result['segments'], srt_counter))
            srt_counter = len(srt_results) + 1

    # Sonuçları belirtilen dosya yoluna kaydet
    with open(output_txt_path, "w", encoding="utf-8") as file:
        for result in txt_results:
            file.write(result + "\n\n")

    with open(output_srt_path, "w", encoding="utf-8") as file:
        for result in srt_results:
            file.write(result)

    print(f"Transcriptions saved to {output_txt_path} and {output_srt_path}")

def generate_srt(segments, start_counter):
    """
    Segmentlerden SRT dosyası oluşturur.

    :param segments: Zaman damgalı segmentlerin listesi
    :param start_counter: SRT giriş numarasını başlatmak için sayaç
    :return: SRT formatında string listesi
    """
    srt_entries = []
    for i, segment in enumerate(segments, start=start_counter):
        start_time = format_timestamp(segment['start'])
        end_time = format_timestamp(segment['end'])
        text = segment['text'].strip()
        srt_entry = f"{i}\n{start_time} --> {end_time}\n{text}\n\n"
        srt_entries.append(srt_entry)
    return srt_entries

def format_timestamp(seconds):
    """
    Zaman damgalarını SRT formatına dönüştürür.

    :param seconds: Saniye cinsinden zaman
    :return: SRT formatında zaman stringi
    """
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

# Kullanım örneği
audio_files = [
    "/content/drive/MyDrive/AI Homework/Y2meta.app-Yapay Zekâ Hakkında _ Take Off İstanbul 2023-(1080p).mp4",
    "/content/drive/MyDrive/AI Homework/Y2meta.app-Yapay Zekâ Hakkında _ Take Off İstanbul 2023-(1080p).mp3",
    "/content/drive/MyDrive/AI Homework/Y2meta.app-Yapay Zekâ Hakkında _ Take Off İstanbul 2023-(1080p).wav"
]
output_txt_file = "/content/drive/MyDrive/AI Homework/sonuçlar.txt"
output_srt_file = "/content/drive/MyDrive/AI Homework/sonuçlar.srt"

transcribe_audio_files(audio_files, output_txt_file, output_srt_file)


MoviePy - Writing audio in temp_audio.wav


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



MoviePy - Done.


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcriptions saved to /content/drive/MyDrive/AI Homework/sonuçlar.txt and /content/drive/MyDrive/AI Homework/sonuçlar.srt
